In [1]:
from winevt import EventLog

events = []

def handleEvent(action, pContext, event):
    evt = {}
    if hasattr(event, "EventData"):
        if hasattr(event.EventData, "Data"):
            if event.EventData.Data[1].cdata == "Furqan Saeed":
                evt["EventID"] = event.EventID
                for item in event.EventData.Data:
                    evt[item["Name"]] = item.cdata
                events.append(evt)

cb = EventLog.Subscribe("Security", "Event/System[EventID=4663 or EventID=4660]", handleEvent)

In [24]:
import win32evtlog

num = 0;
hand=win32evtlog.OpenEventLog(None, "Security")
flags= win32evtlog.EVENTLOG_BACKWARDS_READ|win32evtlog.EVENTLOG_SEQUENTIAL_READ

while(1):
    records = win32evtlog.ReadEventLog(hand, flags, 0)
    # print(dir(records[0]))
    if not records:
        break;
    for record in records:
        if record.EventID == 4663:
            print(str(record.EventID) + " " + record.SourceName + " " + str(record.ComputerName) + " " + record.TimeWritten.Format())
            num = num + 1
    
win32evtlog.CloseEventLog(hand)
print(num)

4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:57:10 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:57:10 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:57:09 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:57:09 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:56:55 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:56:54 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:56:54 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:56:54 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:56:42 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:56:42 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:56:42 2021
4663 Microsoft-Windows-Security-Auditing DESKTOP-LCHJIQP Mon Nov 22 19:56:42 2021
4663 Microsoft-W